# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib

In [ ]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="Detection")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
# TODO: Create compute cluster

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
path = 'https://raw.githubusercontent.com/Tekhunt/ML-Project/master/loands.csv'
data = TabularDatasetFactory.from_delimited_files(path = path)
data.to_pandas_dataframe().head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
       "n_cross_validations": 3,
       "enable_early_stopping": True,
       "experiment_timeout_minutes" :15,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       "verbosity": logging.INFO,
   }

# TODO: Puyour experiment name here


automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    task='classification',
    primary_metric= 'accuracy',
    training_data= data,
    label_column_name= 'Loan_Status',
    n_cross_validations=5)

In [ ]:
# TODo your experiment name here
#remote_run = experiment.submit(automl_config)

remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion()

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
parameter_values = best_run.get_details()['runDefinition']['arguments']


In [ ]:
# Retrieve the best automl model.
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#Register the model
description ='loan prediction model'
model_name='loan-prediction-model'
model_path='./'
tags = None
model = remote_run.register_model(model_name = model_name, description = description , tags = tags)
print(auto_ml_run.model_id)

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Save the best model for the deployement
import os
os.makedirs('./remote_model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./remote_model','best_remote_model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./remote_model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'loan-prediction-model', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description=''
)

In [ ]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [ ]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_remote_run.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_remote_run.py',
                                    environment=myenv)

In [ ]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [ ]:
# deploye the model as a web service
service_name =  'loan-prediction'
service = Model.deploy(workspace=ws, 
                       name= service_name,
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
service

In [ ]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

In [ ]:
x_data_sample = data.take(3)
y_data_sample = x_data_sample.pop('Loan_Status')

#x_data_sample['Column1'] = 0.0

# convert the records to a json data file
json_data = x_data_sample.to_json()#'records'

scoring_data = json.dumps({'data': json_data})
print(scoring_data)

In [ ]:

output = service.run(scoring_data)

In [ ]:
print(output)

In [ ]:
print(y_data_sample)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:

service.delete()
aml_compute.delete()